# Image classification using neural networks

Deep neural networks are the current best approach in artificial intelligence and machine learning systems. Machine learning is often called "deep learning" because the learning happens in deep neural networks, with many artificial neurons between the input and the output.

A neural network is based on an analogue of the brain, with _neurons_ (nerve cells) connected to each other in a _network_. Each neuron takes a bunch of inputs (whether from the inputs to the network or from other neurons), adds them up, and uses that to generate an output. Those outputs then feed into either other neurons or the output of the network as a whole.

Each neuron has several parameters. Each input to the neuron is weighted, and it's these weighted inputs that are added. Each neuron also has a bias, that acts as a threshold value for whether it generates an output.

These neurons are typically organised in _layers_, with neurons in one layer feeding forward into the next. The diagram below shows a network of a few _dense_ layers, where each input to the layer feeds into each neuron.

![A sample neural network](https://github.com/NeilNjae/ou-click-start-ai/blob/main/4.image-classification/pic/sample_neural_network.png?raw=1)

To do image classification with this type of neural network, each pixel in the image is one input. The value of each pixel feeds into each neuron in the first layer, and so on through the network. The final layer has one neuron per class the network knows about, and the values at these outputs is the probability that this image is an example of each of the classes.

When it's first created, the network has random weights throughout, so generates random classifications for each input. This is generally not what we want.

To train the network, we need a large collection of images, each with its _label_ of the true class for that image. During training, we feed in an image and compare the output generated by the network to the output we want (the image's label). We can then see how the network's actual output differs from the desired output, and find the _error_ (difference) between them. The actual output is a combination of the weights and inputs of the final layer of neurons. We can combine the error and the weights to work out how to adjust the weights going into the output layer, and how we should change the values on the neurons that feed into that layer. That gives us an error in the next-to-last layer of neurons; we can use that to adjust the weights feeding into that layer, and so on back to the input layer.

We then the next image from our training data and adjust the weights again.

Once we've done all the images in the training set, we've completed one _epoch_ of training and the network is a bit better at classifying images.

We then go through another epoch of training and the network hopefully improves a bit more. As we do more training, we get to the point of diminishing returns and we can stop.

Enough talk. Let's build a neural network.

## Important: before you start
Before you run any cells in this notebook, you need to ensure you're using a GPU. This will massively increase the speed of using the neural networks.

Click on the Colab menu "Runtime". Select "Change runtime type". Select the "T4" GPU. This will restart the notebook, so you'll have lost any work you've already done.

Once you're connected to a T4 runtime (you can see by looking at the top-right of the window, next to the "RAM" and "Disk" traces), you can start work in this noteobok.

# Loading some data

We need to train the neural network on some data, so we'll load some. We'll use the [standard CIFAR-10 set of small images](https://www.cs.toronto.edu/~kriz/cifar.html), in ten classes.

First, we import the libraries we'll use.

In [ ]:
# Load some toolkits we will need later
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, optimizers, metrics, Sequential
from tensorflow.keras.layers import *

import tensorflow_datasets as tfds

We set some parameters then load the data.

In [ ]:
ROWS = 32
COLS = 32
CHANNELS = 3
INPUT_SHAPE = (ROWS, COLS, CHANNELS)

BATCH_SIZE = 64

In [ ]:
# Loading the data
(train_data, test_data), dataset_info = tfds.load('cifar10',
    split=['train', 'test'],
    with_info=True)
train_data, test_data, dataset_info

We'll reformat the dataset into the form we need for the network to use. We split the data into a _training_ set, that we use to train the network; a _validation_ set, that we use to keep an eye on training as it progresses, and a _test_ set for our final evaluation.

In [ ]:
# Preparing the data

# define the list of class labels
class_names =  ['airplane', 'automobile', 'bird', 'cat', 'deer',
                 'dog', 'frog', 'horse', 'ship', 'truck']

class_num = len(class_names)

def ds_elem_transform(elem):
    return (tf.cast(elem['image'], tf.float32) / 255, # convert pixel values to range 0-1
           tf.one_hot(elem['label'], 10) # one-hot encoding for labels, 10 choices
           )

# Transform every element of the dataset, rename the result.
train_validation_data = train_data.map(
    ds_elem_transform, num_parallel_calls=tf.data.AUTOTUNE)

# Preparing the training and validation data
# Take the elements at index 9, 19, 29... into the validation dataset
validation_data = train_validation_data.shard(10, 9)
validation_data = validation_data.batch(64)
validation_data = validation_data.cache()
validation_data = validation_data.prefetch(tf.data.AUTOTUNE)

# Create a new train_data dataset. Append the remaining shards of the train_validation_data dataset.
train_data = train_validation_data.shard(10, 8)
for i in range(8):
    train_data = train_data.concatenate(train_validation_data.shard(10, i))

train_data = train_data.cache()
train_data = train_data.shuffle(dataset_info.splits['train'].num_examples)
train_data = train_data.batch(64)
train_data = train_data.prefetch(tf.data.AUTOTUNE)

# Perform the same steps on the test dataset:
# transform each element, batch, cache, and prefetch.
test_data = test_data.map(
    ds_elem_transform, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(64)
test_data = test_data.cache()
test_data = test_data.prefetch(tf.data.AUTOTUNE)

How much data do we have?

In [ ]:
len(train_data), len(validation_data), len(test_data)

That's 704 _batches_ of training data, with 64 images per batch, meaning there are just over 45,000 images to train from.

### Viewing some pictures

We use the `matploblib` library to show 25 sample images in the `train_data` we have loaded.

In [ ]:
# Get the first batch.
# (This may take a moment as it reads and caches the Dataset)
sample_imgs, sample_labels = train_data.as_numpy_iterator().next()

# The canvas size is first initialized by setting the parameter figsize in function plt.figure()
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)  # draw five rows and five columns in canvas
    plt.imshow(sample_imgs[i], cmap=plt.cm.binary)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title(class_names[np.argmax(sample_labels[i])])
plt.show()

# Implementing a neural network

We now implement a straightforward neural network. We have an input layer, a `Flatten` layer that converts the 32×32×3 image into a set of 3027 values, and two `Dense` layers to do the classification.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(INPUT_SHAPE),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='sigmoid'),
    tf.keras.layers.Dense(class_num, activation='softmax')
])
model.summary()

The summary shows the structure of the network and how many parameters it has. In this case, just over three-quarters of a million.

We `compile` the model with some parameters to control how training progresses, then we `fit` the model to the training data. The validation data isn't used to train the network. Instead, after each epoch of training, we evalaute the model on the validation data to see how well it performs on other data. This helps when it comes to overfitting, as we'll see later.

In [ ]:
model.compile(
    'SGD',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_data,
    validation_data=validation_data,
    epochs=10,
         )

We can see how the accuracy of the model changes over training. We can also see that the accuracy when using the validation data is similar to the training data.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))
plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()  # Automatic detection of elements to be shown in the legend
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.figure()

### Making predictions on test data

We can now use the model to make predictions on the test data.

In [ ]:
test_predictions = model.predict(test_data)
test_predictions.shape

This gives us 10,000 predictions across ten classes. If we look at one of the predictions, we can see the probabilities of the different classes.

In [ ]:
test_predictions[1]

The predicted label is the index of the highest-valued output for each prediction.

In [ ]:
predict_labels = np.argmax(test_predictions, axis=1)

Now we view some images and give the predicted and actual labels for each.

In [ ]:
# View the true and predicted labels of sample images
plt.figure(figsize=(15,10))
test_imgs, test_labels = test_data.as_numpy_iterator().next()

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_imgs[i], cmap=plt.cm.binary)
    p_class = predict_labels[i]
    a_class = np.argmax(test_labels[i])
    plt.title(f"P: {class_names[p_class]} (A: {class_names[a_class]})",
                                  color=("green" if p_class == a_class else "red"))
plt.show()


Finally, we can evaluate the overall accuracy of the model on test data.

In [ ]:
model_results = model.evaluate(test_data, return_dict=True)
model_results

Your results will vary, as your neural network was given its own random weights at the start. But you should see an accuracy of around 35–40%.

# Why use CNN?

The neural network above works, but not very well. Just under 40% accuracy, from a range on only ten classes, isn't great.

One reason for this is that network throws away all the spatial information in that first `Flatten` layer. Subsequent layers no longer know about which pixels are nearby to other pixels in the original image. A better approach would be to have a neural network that looks at just small regions of the image and feeds that to the next layers.

That is a convolutional neural network.

## Convolution layer

The idea of the convolution layer is to create a filter (also called kernel). The filter is used to scan across the image and create a representation of the image corresponding to the filter. In this way, we can think of the filter as a specific feature extraction mechanism for the image. For instance, a filter may detect a block of colour, or a horizontal edge, or a curve.

A single convoution layer may contain many filters (32 filters in the example below). That means the layer can detect 32 features in the image. The network learns which features are important during training.

![Example filters](https://github.com/NeilNjae/ou-click-start-ai/blob/main/4.image-classification/pic/Example-filters.png?raw=1)

If we use 32 different filters in the convolution layer, then we create 32 different representations, called **feature maps**, of the input image. These different feature maps in combination can help us identify the input image correctly.

How a filter operates is illustrated below. Simply speaking, we overlay a filter, which is a small matrix of weights, on top of the input matrix, e.g. starting from the top-left corner and then sliding from left to right and from top to bottom.

![An example convolution](https://github.com/NeilNjae/ou-click-start-ai/blob/main/4.image-classification/pic/example-convolution.png?raw=1)

The **filter** shown in the example is of size *3×3*. It is applied on a *5×5* input matrix (you can consider it as the matrix of pixels for an image). When the filter moves through the image it does its computation in each position, then moves to cover another *3×3* part of the input. The right-hand side matrix shows the output of each step.

*The values in the kernel are learnt during the training step.*

# Building a CNN model
Let's use a convolution layer for image recognition.

We'll use a convolution layer that has 32 filters. We then `Flatten` those neurons into a set of 33,000 neurons, and use a couple of `Dense` layers to use those features and create the image classification.

![A simple CNN](https://github.com/NeilNjae/ou-click-start-ai/blob/main/4.image-classification/pic/cnn-diagram-simple.png?raw=1)

In [ ]:
model2 = Sequential([
    Input(INPUT_SHAPE),
    Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    # MaxPooling2D(pool_size=(2, 2)),
    # Dropout(0.5),
    Flatten(),
    Dense(128),
    Dense(class_num, activation='softmax')
])

model2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

model.summary()

Again, we compile and train the model.

In [ ]:
model2.compile(
    'adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model2.fit(train_data,
    validation_data=validation_data,
    epochs=10,
         )

There's good news and bad news.

The good news is that the accuracy is much better than when not using the convolution layer. The training accuracy will be about 85%, the validation accuracy about 55%.

The bad news is that difference between the performance on the training and validation data. It's clearer if we plot the training history.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))
plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()  # Automatic detection of elements to be shown in the legend
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.figure()

This shows the model is _overfitting_: it's getting very good at classifying the training data, but does much less well on data it's not been trained on. (The validation data is used to check how well the model works, not to improve the model's performance.)

## Addressing overfitting

We can address overfitting by throwing away some of the information _inside_ the model during training. We do this in two ways: pooling and dropout.

Pooling uses the observation that pieces of the image near each other are very similar, and will have very similar feature maps from a convolutional layer. In any given small region ("pool") of a feature map, we can take just one pixel to go forward. Generally, we take the largest value in the pool.

Dropout just plain discards information during training. Networks can overfit if all the weights can be adjusted together. During training, a dropout layer will randomly turn off some of the connections with each training example. This allows the weights to change independently and therefore makes the network more robust.

By combining both pooling and dropout layers, we can afford to make the network deeper before overfitting occurs. We've done this in the model below, where we have three convolution layers.

In [ ]:
model3 = Sequential([
    Input((32, 32, 3)),
    Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(class_num, activation='softmax')
])

model3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [ ]:
model3.summary()

Note that the last feature map is only 4×4 across, but has 64 features to choose from.

## Exercise

We can now compile and train the model, this time for **30** epochs. Apart from the number of epochs, use exactly the same code as you did above for `model2`.

### Solution

In [ ]:
history = model3.fit(train_data,
    validation_data=validation_data,
    epochs=30
         )

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))
plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.figure();

### End of solution

This seems to have gone much better. The accuracy is better, and the accuracy of the validation dataset is very close to the training dataset.

How does it work on the test dataset?

In [ ]:
model3_results = model3.evaluate(test_data, return_dict=True)
model3_results

Your results will vary, but you'll probably see an accuracy in the 70–75% range.

We can show some images with their predicted and actual labels.

In [ ]:
test_predictions = model3.predict(test_data)

predict_labels = np.argmax(test_predictions, axis=1)

# View the true and predicted labels of some sample images

plt.figure(figsize=(15,10))
test_imgs, test_labels = test_data.as_numpy_iterator().next()

for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_imgs[i])
    p_class = predict_labels[i]
    a_class = np.argmax(test_labels[i])

    plt.title(f"P: {class_names[p_class]} (A: {class_names[a_class]})",
                                  color=("green" if p_class == a_class else "red"))
plt.show()

This shows the output of the model, but we can also peer inside it and look at the feature maps it generates.

We do that by creating another "model" that reuses the same trained layers in `model3`. The input to this new model is the same as before, but the model's output is the output of the first convolutional layer.

First we need to check the locations of the convolutional layers in the model.

In [ ]:
list(enumerate(model3.layers))

We now create the model with the same input and first convolutional layer.

In [ ]:
intermediate_layer_model = keras.Model(inputs=model3.inputs,
                                       outputs=model3.layers[0].output)
intermediate_layer_model.summary()

We now look at the feature maps for a particular image. We'll use image 0 from the test batch.

In [ ]:
test_image_number = 0

In [ ]:
# ploting the original image
plt.figure()
plt.imshow(test_imgs[test_image_number])

Now we can show some sample outputs of the intermediate model: the feature maps of this image.

In [ ]:
intermediate_output = intermediate_layer_model(test_imgs)

In [ ]:
plt.figure(figsize=(10, 5))

for i in range(32):
    plt.subplot(4, 8, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(intermediate_output[test_image_number,:,:,i])
    plt.xlabel(f'Filter {i}', fontsize=10)

I've no idea what features your model will find. But you'll probably see some feature maps that indicate foreground or background, and some that detect edges between them in different directions.

These are the features that later stages in the model will work with.

Following the same process, we can look at the output of the second covolutional layer.

In [ ]:
intermediate_layer_model2 = keras.Model(inputs=model3.inputs,
                                       outputs=model3.layers[2].output)
intermediate_layer_model2.summary()

In [ ]:
intermediate_output2 = intermediate_layer_model2(test_imgs)

plt.figure(figsize=(10, 5))

for i in range(32):
    plt.subplot(4, 8, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(intermediate_output2[test_image_number,:,:,i])
    plt.xlabel(f'Filter {i}', fontsize=10)

These features will be more abstract than the ones before, as they're combinations of some or many of the features from the first convolutional layer.

Finally, the third convolutional layer.

In [ ]:
intermediate_layer_model3 = keras.Model(inputs=model3.inputs,
                                       outputs=model3.layers[4].output)
intermediate_layer_model3.summary()

In [ ]:
intermediate_output3 = intermediate_layer_model3(test_imgs)

# Plotting the 8 intermediate feature maps generated by the 8 filters in the first convolution layer

plt.figure(figsize=(10, 5))

for i in range(64):
    plt.subplot(8, 8, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(intermediate_output3[test_image_number,:,:,i])

These will bear very little direct connection to the input image, but should somehow encode important features that help the classification of the image.

In [ ]:
test_image_number = 9

In [ ]:
plt.imshow(test_imgs[test_image_number])

plt.figure(figsize=(10, 5))
for i in range(32):
    plt.subplot(4, 8, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(intermediate_output[test_image_number,:,:,i])
    # plt.xlabel(f'Filter {i}', fontsize=10)

plt.figure(figsize=(10, 5))
for i in range(32):
    plt.subplot(4, 8, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(intermediate_output2[test_image_number,:,:,i])
    # plt.xlabel(f'Filter {i}', fontsize=10)

plt.figure(figsize=(10, 5))
for i in range(64):
    plt.subplot(8, 8, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(intermediate_output3[test_image_number,:,:,i])
    # plt.xlabel(f'Filter {i}', fontsize=10)

# Conclusion

With these activities, you've explored some of the techniques of modern machine learning and artificial intelligence. Deep convolutional neural networks are a mainstay of deployed AI systems and are used for all sorts of image processing tasks. Variations on these ideas can be applied to other types of data, such as understanding speech and text.

You've also looked at how convolutional networks work internally, with the creation of feature maps that encode important parts of the image, and how these feature maps are combined to give higher-level representations of an input.

You've also seen why these systems take a lot of time, computing power, and data to get good performance. The CIFAR dataset is only small, but required many rounds of training, on specific hardware, to get good results.

CNNs are just one technique in machine learning, but the ideas you used here should give you a much better understanding of how these systems work.